# Redes Neurais e sua Implementação

In [236]:
import string
import re
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ruasgar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [237]:
df = pd.read_csv("../data/final_dataset.csv")

In [238]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4',
       'full_article'],
      dtype='object')

## 1 - Features Importantes
- Label_bias (categórica): Indica se o texto foi classificado como enviesado, não-enviesado ou se não foi possível atingir um consenso quanto a classificação.
- Label_opinion (categórica): Indica de que modo o viés se manifesta na percepção dos entrevistados; especificamente separando casos de exposição de opinião do autor ou com fatos que corroborem um viés. (Um pouco fuzzy demais, talvez?)
- Biased_words(vetor): Indica as palavras marcadas como "denunciantes" da presença de viés. 
- Topic(categórica): Indica o assunto do texto, dentro das categorias PREENCHER AQUI   

## 2 - Pré-Processamento

### 2.1 - Básico textual (remoção de stopwords, filtro de expressões com números, etc)

- A priori, vamos considerar como tokens todas as palavras que:
    - tenham mais do que 3 letras
    - não possuam números
    - não estejam nas stopwords do inglês
    
- Vamos remover os acentos e, mediante a escolha do usuário, aplicar um stemmer.
- Visando o mapeamento das palavras para números inteiros, vamos utilizar o padrão de marcar alguns tokens especiais:
    - 0: padding (completar o tamanho das representações)
    - 1: Start-of-Sequence
    - 2: tokens desconhecidos, necessários para novas palavras

In [239]:
digit_pattern = r'\d+(\.\d+)?'
solo_quotations_pattern = pattern = r"^(?:' '|\\" "|\`\`)$"

remove_quotation = r"'\b|\b'\s|\s'\b|\"\b|\b\"\s|\s\"\b|``\b|\b``\s|\s``\b" # como em "America is Great"
remove_symbols_only = r'\b[^\w\s]+\b' # como em '--'
remove_symbols_if_aside = r'\b[^\w\s]|[^\w\s]\b' # como em 'dog-'

remove_patterns = remove_quotation,remove_symbols_if_aside, remove_symbols_only

stop_words = set(stopwords.words('english')) # talvez o set deixe mais rápido

def preprocess_sentences(df):
    
    prepped_sentences = []

    for s in df['sentence']:
        # Remove acentos e põe tudo para lower case
        s = unidecode(s).lower()
        tokens = word_tokenize(s,language="english")

        tokens = [
            re.sub('|'.join(remove_patterns),'',t)
            for t in tokens 
            if not bool(re.search(digit_pattern,t))
            and t not in string.punctuation
            and t not in stop_words
        ]

        tokens = [
            t for t in tokens
            if not bool(re.match(solo_quotations_pattern,t)) 
            and len(t) >= 3
        ]

        prepped_sentences.append(tokens)

    df['sentence'] = prepped_sentences

    return df 

df_functions_test_copy = df.copy()
df_functions_test_copy = preprocess_sentences(df_functions_test_copy)
df_functions_test_copy['sentence'].head()


0    [youtube, making, clear, birtherism, platform,...
1    [increasingly, bitter, dispute, american, wome...
2    [may, humanitarian, crisis, driving, vulnerabl...
3    [professor, teaches, climate, change, classes,...
4    [world, antidoping, agency, tuesday, said, rus...
Name: sentence, dtype: object

### 2.1 - Stemmers

- Temos o objetivo de avaliar como diferentes estratégias de stemming afetam os resultados do treinamento. A função a seguir visa abstrair esse passo para um etapa separada de pré-processamento. 
- Serão testados os stemmer de Porter e de Lancaster, além da lematização via WordNet. Vamos também obter os resultados do modelo quando utilizados sem qualquer stemmer. 

In [240]:
def apply_stemmer(sentence, stemmer_type:str):

    # Função a ser executada para cada sentença, com expectativa de uso em algum filtro, por exemplo

    stemmer = {}
    lemmatizer = {}

    if stemmer_type == "Porter":
        stemmer = PorterStemmer()
        return [stemmer.stem(token) for token in sentence]

    elif stemmer_type == "Lancaster":
        stemmer = LancasterStemmer()
        return [stemmer.stem(token) for token in sentence]
    
    else:  # "Wordnet":
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.stem(token) for token in sentence]


- Aplicando stemmer para teste da função 

In [241]:
df_functions_test_copy['sentence'] = df_functions_test_copy['sentence'].apply(lambda x: apply_stemmer(x,"Porter"))
df_functions_test_copy['sentence'].head()

0    [youtub, make, clear, birther, platform, year,...
1    [increasingli, bitter, disput, american, women...
2    [may, humanitarian, crisi, drive, vulner, peop...
3    [professor, teach, climat, chang, class, subje...
4    [world, antidop, agenc, tuesday, said, russian...
Name: sentence, dtype: object

### 2.3 - Separando o dataset

In [242]:
def train_valid_test_split(features, labels):
    """ Retorna uma lista de tuplas contendo os datasets de features e de labels para cada segmento (treino, validação, teste) """
    
    # Treino-val e Teste
    shuffle_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=892)
    train_val_indexes, test_indexes = next(shuffle_train_test.split(features.values, labels.values))
    train_val_df, train_val_labels = features.iloc[train_val_indexes], labels.iloc[train_val_indexes]
    test_df, test_labels = features.iloc[test_indexes], labels.iloc[test_indexes]

    # Treino e Validação
    shuffle_train_validate = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=124)
    train_indexes, validation_indexes = next(shuffle_train_validate.split(train_val_df.values,train_val_labels.values))
    train_df, train_labels = features.iloc[train_indexes], labels.iloc[train_indexes]
    validation_df, validation_labels = features.iloc[validation_indexes], labels.iloc[validation_indexes]


    return [(train_df, train_labels), (validation_df, validation_labels), (test_df, test_labels)]


In [243]:
train, val, test = train_valid_test_split(df_functions_test_copy,df_functions_test_copy['type'])
train[0].shape

(848, 12)

### 2.4 - Encoding (palavras)

- Para representar as palavras, vamos criar um encoding com base na frequência apresentada no corpus.
- Quando formos alimentar a rede neural, vamos utilizar uma camada de WordEmbedding do TensorFlow para representação do espaço "textual".

In [245]:
train_df_copy = train[0]

vocab = Counter()
for sentence in train_df_copy['sentence']:  
    vocab.update(sentence)

words = tf.constant([word for word,_ in vocab])
words_ids = tf.range(len(vocab), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words,words_ids)
num_oov_buckets = 5000
table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)


ValueError: too many values to unpack (expected 2)

- Convertendo os tokens para o índice na frequência (tanto da sentença, como das palavras que denotam viés)

In [ ]:
def convert_words_to_freq(data, vocab):

    for index,row in data.iterrows():

        sentence = row['sentence']
        biased_words = row['biased_words4']

        sentence = list(map(lambda word:int(vocab[word]),sentence))
        biased_words = list(map(lambda word:int(vocab[word]),biased_words))

        data.at[index,'sentence'] = tf.constant(sentence)
        data.at[index, 'biased_words4'] = tf.constant(biased_words)

    return data

train_df_copy = convert_words_to_freq(train_df_copy, vocab)
train_df_copy.head(1)

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4,full_article
867,"(tf.Tensor(38, shape=(), dtype=int32), tf.Tens...",https://thefederalist.com/2019/08/13/need-chri...,federalist,white-nationalism,right,52,1,Biased,Somewhat factual but also opinionated,Our religious liberty never proceeded from att...,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso...",('The House Ways and Means Committee will hold...


In [ ]:
def add_padding(data):

    data['sentence'] = tf.keras.preprocessing.sequence.pad_sequences(
                            data['sentence'],  
                            value=[0]
                        )
    return data

train_df_copy = add_padding(train_df_copy)

/tmp/ipykernel_4597/557190795.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentence'] = tf.keras.preprocessing.sequence.pad_sequences(


In [ ]:
train_df_copy.head()

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4,full_article
867,0,https://thefederalist.com/2019/08/13/need-chri...,federalist,white-nationalism,right,52,1,Biased,Somewhat factual but also opinionated,Our religious liberty never proceeded from att...,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso...",('The House Ways and Means Committee will hold...
692,0,https://www.alternet.org/2019/09/everything-yo...,alternet,sport,left,41,1,Non-biased,Entirely factual,"Since 1983, Kim Karsh has helped baseball team...","(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso...",('There have been at least 555 confirmed cases...
698,0,https://www.reuters.com/article/climatechange-...,reuters,environment,center,41,1,Non-biased,Entirely factual,"COPENHAGEN/OSLO, March 10 (Reuters) - The oil ...","(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso...",('A recent study found that college students f...
416,0,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,25,1,Biased,Somewhat factual but also opinionated,Speaking to the country for the first time fro...,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso...",('President Obama has ordered the Department o...
346,0,https://www.foxnews.com/health/experts-warn-of...,fox-news,sport,right,21,1,Non-biased,Entirely factual,David Ayala was 11 years old when he began com...,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso...",('House Republicans are gearing up for a susta...
